<a href="https://colab.research.google.com/github/StephenSheng1101/DMAsgmnt/blob/main/Ridge_Regression_Model_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Ridge Regression**


**Data Preprocessing before begin with Ridge Regression**

In [10]:
from pathlib import Path
import pandas as pd
import numpy as np
import tarfile
import urllib.request
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import StandardScaler

In [11]:
transport = pd.read_csv("dataset.csv")
transport.describe()
transport.info()

ParserError: ignored

In [ ]:
transport.isnull().sum()

In [ ]:
#handling missing value
transport["departure_long"].fillna(transport["departure_long"].mean(), inplace=True)
transport["departure_lat"].fillna(transport["departure_lat"].mean(), inplace=True)
transport["occupancy"].fillna(transport["occupancy"].mode()[0], inplace=True)
transport.drop(columns=['ID'], inplace=True)

# Drop rows with missing "departure_time","arrival_long" and "arrival_lat"
transport.dropna(subset=["departure_time", "arrival_long", "arrival_lat"],inplace = True)
transport.isnull().sum()

# Drop rows with a 0 value of occupancy
transport = transport.drop(transport[transport['occupancy'] <= 0].index)

# Handling Outliers
# Select columns for numeric attributes
num_attribs = ['departure_long', 'departure_lat', 'arrival_long', 'arrival_lat', 'occupancy', 'travel_fee']

# Remove outliers using Z-score method
z_scores = np.abs((transport[num_attribs] - transport[num_attribs].mean()) / transport[num_attribs].std())
transport = transport[(z_scores < 3).all(axis=1)]
transport.info()
transport

In [ ]:
transport.head()

In [ ]:
# Convert departure_time to datetime
transport['departure_time'] = pd.to_datetime(transport['departure_time'])

# Extract features from departure_time
transport['hour_of_day'] = transport['departure_time'].dt.hour
transport['day_of_week'] = transport['departure_time'].dt.dayofweek

In [ ]:
travelfee = train_set.drop("travel_fee", axis=1)
travelfee_labels = train_set["travel_fee"].copy()

In [ ]:
from scipy.spatial.distance import euclidean
# Calculate Euclidean distance between departure and arrival locations
transport['distance'] = transport.apply(lambda row: euclidean((row['departure_lat'], row['departure_long']),
                                                             (row['arrival_lat'], row['arrival_long'])), axis=1)


In [ ]:
from sklearn.model_selection import train_test_split


train_set, test_set = train_test_split(transport.head(1000000), test_size=0.2, random_state=42)

print("Length of train_set:", len(train_set))
print("Length of test_set:", len(test_set))

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
# Select features and target
X = train_set[['occupancy', 'distance']]
y = train_set['travel_fee']

# Preprocessing pipeline
num_attribs = ['occupancy', 'distance']
num_pipeline = make_pipeline(
    StandardScaler()  # Scale features
)
preprocessing = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
    # Apply numeric pipeline to numeric attributes
])

In [ ]:
transport_prepared_2 = preprocessing.fit_transform(transport)
transport_prepared_2

**Linear Regression**


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
X2 = transport_prepared_2  # Features
y2 = transport['travel_fee']  # Target

In [ ]:
# Split data into training and testing sets
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=42)

In [ ]:
from sklearn.linear_model import LinearRegression
mlr = LinearRegression()
mlr.fit(X2, y2)

In [ ]:
y_pred_linear = mlr.predict(X2)
y_pred_linear [0:5]

In [ ]:
mlr.intercept_

In [ ]:
mlr.coef_

In [ ]:
mlr_score = cross_val_score(mlr, X2_train, y2_train, scoring=None,verbose=2)
print(mlr_score)
print(mlr_score.mean(),mlr_score.std())

**Select and Train a Model on the Prepared Training Set**

**Ridge Regression**

Ridge Regression is a regularization technique used to prevent overfitting in linear regression models by adding a penalty term to the cost function based on the L2 norm of the coefficients. Let's continue with implementing Ridge Regression the preprocessed data.

In [ ]:
from sklearn.linear_model import Ridge
# Create and train Ridge Regression model
ridge_model = Ridge(alpha=1.0)
ridge_model.fit(X2_train, y2_train)

In [ ]:
# Predict travel fees on test set
y_pred_ridge = ridge_model.predict(X2_test)
print("predicted: ",y_pred_ridge)

In [ ]:
ridge_score = cross_val_score(mlr, X2_train, y2_train, cv=5,verbose=2)
print("Ridge Score: ", ridge_score)
print(ridge_score.mean(),ridge_score.std())

# Fine-tune Ridge Regression model (shortlisted promising models) using crossvalidation.


In [ ]:
from sklearn.model_selection import GridSearchCV

parameters = {"fit_intercept":[True, False],'n_jobs':[None,1],'positive':[True,False]}

mlr = LinearRegression()
mlr_cv = GridSearchCV(estimator = mlr, param_grid = parameters ,cv = 5)
mlr_cv.fit(X2,y2)
print("Tuner hyperparameters: (best parameters)", mlr_cv.best_params_)
print("Accuracy", mlr_cv.best_score_)
best_mlr = mlr_cv.best_estimator_

In [ ]:
parameters = {"alpha":[0.001,0.01,0.02,0.03,0.04,0.05,0.06,0.08,1,2,3,5,8,10,20,50,100]}

ridge_cv = GridSearchCV(Ridge(),parameters,scoring='neg_mean_squared_error',cv = 5)
ridge_cv.fit(X2,y2)
print("Tuner hyperparameters: (best parameters)", ridge_cv.best_params_)
print("Accuracy", ridge_cv.best_score_)

best_ridge = ridge_cv.best_estimator_

In [ ]:
from sklearn.ensemble import VotingRegressor

voting_reg = VotingRegressor(
    estimators=[('mlr', best_mlr),('ridge', best_ridge)]
)
voting_reg.fit(X2,y2)

In [ ]:
voting_reg.fit(X2,y2)

Evaluate on the Test Set for both Models

In [ ]:
mlr = LinearRegression(fit_intercept=False, n_jobs= None, positive= False)
mlr.fit(X2, y2)
y_pred_linear = mlr_cv.predict(X2_test)

In [ ]:
print(pd.DataFrame({'Actual Value':y2_test,'Predicted Values':y_pred_linear}))

In [ ]:
ridge = Ridge(alpha=100)
ridge.fit(X2,y2)
y_pred_ridge = ridge_cv.predict(X2_test)

In [ ]:
print(pd.DataFrame({'Actual Value':y2_test,'Predicted Values':y_pred_ridge}))